# 🌟 Universal Public Data MCP Server - Comprehensive Demo

Welcome to the complete hands-on tutorial! This notebook demonstrates real-time data access from multiple APIs.

## 🚀 What You'll Learn

- 💰 **Real-time financial data** from Yahoo Finance and CoinGecko
- 📰 **Breaking news feeds** from technology RSS sources  
- 🌤️ **Weather data** from wttr.in API
- 📊 **Portfolio analysis** with live market data
- 🛠️ **Production patterns** for error handling and data processing

**Goal**: Master real API integrations with production-ready code patterns

In [ ]:
# Install dependencies for Google Colab
!pip install yfinance>=0.2.0 httpx>=0.25.0 feedparser>=6.0.0 beautifulsoup4>=4.12.0 requests>=2.31.0 -q

print("✅ All dependencies installed successfully!")

In [ ]:
# Core imports
import yfinance as yf
import requests
import feedparser
from datetime import datetime
import json
from typing import Dict, List, Any

print("✅ All imports successful!")
print(f"🕐 Tutorial started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 💰 Real-Time Financial Data

Let's start with **real-time Apple stock data** from Yahoo Finance!

In [ ]:
def get_stock_data(symbol: str) -> Dict[str, Any]:
    """Get real-time stock data from Yahoo Finance"""
    try:
        ticker = yf.Ticker(symbol)
        info = ticker.info
        hist = ticker.history(period="1d")
        
        if hist.empty:
            return {'error': f'No data found for {symbol}'}
        
        current_price = hist['Close'].iloc[-1]
        
        return {
            'symbol': symbol,
            'company_name': info.get('longName', 'N/A'),
            'current_price': round(current_price, 2),
            'market_cap': info.get('marketCap', 0),
            'volume': info.get('volume', 0),
            '52w_high': info.get('fiftyTwoWeekHigh', 0),
            '52w_low': info.get('fiftyTwoWeekLow', 0),
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
    except Exception as e:
        return {'error': str(e)}

# Get Apple stock data
print("🔄 Fetching real-time Apple stock data...")
apple_data = get_stock_data('AAPL')

if 'error' not in apple_data:
    print("\n🍎 APPLE INC. (AAPL) - LIVE DATA")
    print("=" * 40)
    print(f"💰 Current Price: ${apple_data['current_price']:,.2f}")
    print(f"📈 52W High: ${apple_data['52w_high']:,.2f}")
    print(f"📉 52W Low: ${apple_data['52w_low']:,.2f}")
    print(f"🏢 Market Cap: ${apple_data['market_cap']:,}")
    print(f"📊 Volume: {apple_data['volume']:,}")
    print(f"🕐 Updated: {apple_data['timestamp']}")
    print("\n✨ Real data from Yahoo Finance API!")
else:
    print(f"❌ Error: {apple_data['error']}")

## 📊 Portfolio Analysis

Let's create a **multi-stock portfolio** with live data:

In [ ]:
# Define portfolio
symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA']
shares = [100, 50, 75, 25]

portfolio = []
total_value = 0

print("🔄 Fetching portfolio data...")

for symbol, share_count in zip(symbols, shares):
    stock_data = get_stock_data(symbol)
    if 'error' not in stock_data:
        position_value = stock_data['current_price'] * share_count
        total_value += position_value
        
        portfolio.append({
            'symbol': symbol,
            'shares': share_count,
            'price': stock_data['current_price'],
            'value': position_value,
            'company': stock_data['company_name']
        })

# Calculate percentages
for position in portfolio:
    position['percentage'] = (position['value'] / total_value) * 100

print("\n📊 PORTFOLIO ANALYSIS - LIVE DATA")
print("=" * 50)

for position in portfolio:
    print(f"📈 {position['symbol']}: {position['shares']} shares @ ${position['price']:.2f} = ${position['value']:,.2f} ({position['percentage']:.1f}%)")

print(f"\n💰 Total Portfolio Value: ${total_value:,.2f}")
print(f"🕐 Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n✨ Real portfolio data from Yahoo Finance!")

## 📰 Breaking News

Let's fetch **real-time technology news** from RSS feeds:

In [ ]:
def get_tech_news(max_articles: int = 5):
    """Get breaking technology news from RSS feeds"""
    news_sources = [
        {'name': 'TechCrunch', 'url': 'https://techcrunch.com/feed/'},
        {'name': 'Ars Technica', 'url': 'https://feeds.arstechnica.com/arstechnica/index'}
    ]
    
    all_articles = []
    
    for source in news_sources:
        try:
            feed = feedparser.parse(source['url'])
            
            for entry in feed.entries[:max_articles]:
                all_articles.append({
                    'title': entry.title,
                    'source': source['name'],
                    'link': entry.link,
                    'published': entry.get('published', 'N/A'),
                    'summary': entry.get('summary', 'No summary available')[:200] + '...'
                })
        except Exception as e:
            print(f"⚠️ Failed to fetch from {source['name']}: {e}")
    
    return all_articles[:max_articles]

# Get breaking tech news
print("🔄 Fetching breaking technology news...")
news_articles = get_tech_news(5)

print("\n📰 BREAKING TECHNOLOGY NEWS")
print("=" * 50)

for i, article in enumerate(news_articles, 1):
    print(f"\n{i}. 📰 {article['title']}")
    print(f"   🏢 Source: {article['source']}")
    print(f"   🕐 Published: {article['published']}")
    print(f"   🔗 Link: {article['link']}")

print(f"\n📊 Total Articles: {len(news_articles)}")
print(f"🕐 Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n✨ Real news from live RSS feeds!")

## 🌤️ Weather Data

Let's get **real-time weather** for multiple cities:

In [ ]:
def get_weather_data(city: str):
    """Get weather data from wttr.in API"""
    try:
        url = f"https://wttr.in/{city}?format=j1"
        
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        current = data['current_condition'][0]
        
        return {
            'city': city,
            'temperature_c': current['temp_C'],
            'temperature_f': current['temp_F'],
            'condition': current['weatherDesc'][0]['value'],
            'humidity': current['humidity'],
            'wind_speed_kmh': current['windspeedKmph']
        }
    except Exception as e:
        return {'error': str(e)}

# Get weather for multiple cities
cities = ['New York', 'London', 'Tokyo', 'Sydney']

print("🔄 Fetching weather data for multiple cities...")
print("\n🌤️ GLOBAL WEATHER CONDITIONS")
print("=" * 50)

for city in cities:
    weather = get_weather_data(city)
    
    if 'error' not in weather:
        print(f"\n🌍 {weather['city'].upper()}")
        print(f"   🌡️ Temperature: {weather['temperature_c']}°C ({weather['temperature_f']}°F)")
        print(f"   ☁️ Condition: {weather['condition']}")
        print(f"   💧 Humidity: {weather['humidity']}%")
        print(f"   💨 Wind Speed: {weather['wind_speed_kmh']} km/h")
    else:
        print(f"\n❌ {city}: {weather['error']}")

print(f"\n🕐 Weather updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n✨ Real weather data from wttr.in API!")

## 🎯 Summary Dashboard

Let's create a **real-time summary** of all our data:

In [ ]:
print("\n" + "=" * 60)
print("🚀 UNIVERSAL PUBLIC DATA MCP SERVER - LIVE DASHBOARD")
print("=" * 60)

# Quick stock check
quick_stocks = ['AAPL', 'GOOGL', 'MSFT']
print("\n📈 QUICK STOCK PRICES:")
total_quick_value = 0

for symbol in quick_stocks:
    stock = get_stock_data(symbol)
    if 'error' not in stock:
        print(f"   {symbol}: ${stock['current_price']:.2f}")
        total_quick_value += stock['current_price']

print(f"   Combined Value: ${total_quick_value:.2f}")

# Weather summary
print("\n🌤️ WEATHER SNAPSHOT:")
quick_cities = ['New York', 'London']
for city in quick_cities:
    weather = get_weather_data(city)
    if 'error' not in weather:
        print(f"   {city}: {weather['temperature_c']}°C, {weather['condition']}")

print(f"\n🕐 Dashboard Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n✨ ALL DATA IS LIVE FROM REAL APIS - NO MOCKS OR PLACEHOLDERS!")
print("="* 60)

## 🎉 Congratulations!

You've successfully completed the comprehensive tutorial! You've learned how to:

### ✅ **What You Accomplished**
- 💰 **Retrieved real-time financial data** from Yahoo Finance
- 📊 **Built a live portfolio analyzer** with multiple stocks
- 📰 **Fetched breaking news** from technology RSS feeds
- 🌤️ **Gathered weather data** from multiple global cities
- 🎯 **Created a real-time dashboard** combining all data sources

### 🛠️ **Production Patterns You Learned**
- Error handling with try/catch blocks
- Timeout management for API calls
- Data validation and formatting
- Modular function design
- Real-time data aggregation

### 🚀 **About the Full MCP Server**

This tutorial demonstrates a simplified version of the Universal Public Data MCP Server. The complete project includes:

**6 Data Categories**: Financial, News, Government, Geographic, Scientific, Technology

**20+ API Tools**: Each with production-grade features like Redis caching, rate limiting, and comprehensive error handling

**Enterprise Features**: Async processing, intelligent caching, monitoring, and scalable architecture

### 📚 **Next Steps**
- **Explore the full MCP server** at [GitHub Repository](https://github.com/inamdarmihir/universal-public-data-mcp-server)
- **Add more APIs** using the patterns you learned
- **Build visualizations** with matplotlib or plotly
- **Implement caching** for better performance

**🌟 Keep exploring real data and building amazing applications!**